### Ideen Use Case
- Gesetzbuch ZGB
- Kochbuch
- LLM Agent generiert Werbetexe, Social Media Posts

### FWO-Assistent

- Hilft bei der Erstellung von Texten für LinkedIn, Instagram, WhatsApp etc.

- Auf Basis von vergangenen Posts

In [1]:
import os
from langchain_openai import ChatOpenAI

# --- Stelle sicher, dass dieser Key existiert ---
assert "GROQ_API_KEY" in os.environ, "GROQ_API_KEY fehlt in den Env Vars!"

# --- Initialisiere LLM explizit für OpenRouter ---
llm = ChatOpenAI(
    model="openai/gpt-oss-120b",   # wähle ein OpenRouter-Modell, das es wirklich gibt
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
    temperature=0.3,
)

print("Sende Test-Ping...")
try:
    msg = llm.invoke("Sag exakt: pong")
    print("Antworttyp:", type(msg))
    # msg ist i.d.R. ein AIMessage – gib Inhalt sicher aus:
    print("Inhalt:", getattr(msg, "content", msg))
except Exception as e:
    print("FEHLER beim LLM-Aufruf:", repr(e))


c:\Users\amuel\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sende Test-Ping...
Antworttyp: <class 'langchain_core.messages.ai.AIMessage'>
Inhalt: pong


In [2]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI


#LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_MODEL = "openai/gpt-oss-120b"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://api.groq.com/openai/v1"
OPENROUTER_API_KEY = os.getenv("GROQ_API_KEY")
USER_PROMPT="Generiere mir einen Instagrampost"

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

print(type(llm))


<class 'langchain_openai.chat_models.base.ChatOpenAI'>


In [4]:
try:
    print(llm.invoke("Sag nur: pong").content)
except Exception as e:
    print(repr(e))

# Test 2: Env-Variablen sichtbar?
import os
print("OPENAI_API_KEY" in os.environ, os.environ.get("OPENAI_BASE_URL"))
print("GROQ_API_KEY" in os.environ)
print("OPENROUTER_API_KEY" in os.environ)

pong
False None
True
False


In [5]:
from langchain_core.prompts import ChatPromptTemplate

FWO_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "Du bist der FWO-Assistent (Fachschaft Wirtschaft Olten, FHNW). "
     "Bevor du schreibst, beziehst du dich IMMER auf: "
     "1) Kampagnenkonzept (Ziel, Kernbotschaft, Mehrwert) "
     "2) Redaktionsplan (Phase, wiederkehrende Formate, Timing). "
     "Vermeide Genderung und gebrauche bspw. Lehrperson anstatt Lehrer*innen "
     "Generiere jeweils immer zwei Vorschläge zu jeder Plattform (Instagram, WhatsApp, LinkedIn)"
     "Vermeide Floskeln, schreibe aktiv und konkret."
     "If you are unsure or the answer isn't in the context, say that you don't know.\n\n"
     "CONTEXT: \n {context}"
    ),
    ("human","{question}"),
])


In [6]:
# from langchain_core.output_parsers import StrOutputParser

# user_prompt = "Generiere mir einen Post für die PubTour am 16. Oktober"

# chain_simple = (FWO_PROMPT | llm | StrOutputParser())
# result = chain_simple.invoke({"question": user_prompt, "context": ""})
# print(result)
# print(result)

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# List of PDF file paths

pdf_path = "Order pdf/SoMe Konzept.pdf"

loader = PyPDFLoader(str(pdf_path))
pages = loader.load()


# Load all documents
# all_pages_pdf = []
# for pdf in pdf_files:
#     loader = PyPDFLoader(pdf)
#     pages = loader.load()
#     all_pages_pdf.extend(pages)

#print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} pdf documents.")


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = pages

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [9]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

Initial documents: 6
Total chunks: 55
Avg length: 253.0
Min: 48, Max: 297


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


C:\Users\amuel\AppData\Local\Temp\ipykernel_32352\4293025327.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [11]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

['988961e2-6963-4128-8fe8-999f5bf73b7c',
 'a5bad737-7356-4bdc-b75e-f1ffcc4ddd14',
 'f98edf45-659b-4594-8e8d-ba32bf1ca2c6',
 '06baf2a6-408d-41fc-8a5d-bdbdabfb3103',
 'e9043688-d455-408f-98e6-081168d2e18f',
 'c86f75ec-fbac-49f0-b1c6-b4025931d3fd',
 '3d55c071-1f26-4d07-84b0-9368024a4ffd',
 'e12b8f0d-8c8b-4ff4-9394-cef7e381714d',
 '2e68145a-a6ae-4cc7-b848-c06bc2f9ea38',
 '69897cd9-3352-44f9-9a72-b53d1b5ef051',
 '2ccafeec-23fe-4144-b866-c37fe4c76beb',
 '8d477633-c8e1-4752-9118-5eb64c2110f0',
 '2ed72c8c-7db5-4779-8543-57f89791b04e',
 'a3c08db1-8065-4d4e-8711-f96f61250ee3',
 '97bebff9-a426-4a29-839a-a5bcd7a776eb',
 '518d8fa9-285d-4a99-857c-c5e47dfb429d',
 'ec9fcd31-0de5-4b17-94aa-d77af86cf641',
 'c14f10bb-f2d5-488f-af93-4c826b2d5a05',
 'e8ddcdb8-1e69-450e-a0e4-9352784842a3',
 '77d1366b-20ce-4306-8e7e-dfe217d8093b',
 'b180ec81-96ae-4378-9e1c-0a6970298ff4',
 '1ebd97fa-9a79-4246-99ee-7f661e853c27',
 'c80523c8-cfba-4880-927b-87346b03a126',
 '6fce9dc2-fa12-49c6-bad8-e4d06769fd03',
 '1da4b3be-dfc0-

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [13]:
docs = retriever.get_relevant_documents("Hashtags für Instagram")
for i, d in enumerate(docs, 1):
    print(f"--- Retrieved doc {i} ---")
    print(d.page_content[:400], "...\n")

--- Retrieved doc 1 ---
- Grund: Account aktiv halten vor allem in Zeiten, bei denen weniger Posts gemacht 
werden können. 
Hashtags: 
#FHNW #FWO #FachschaftWirtschaftOlten #HappyEaster #HappyChristmas #HappyXmas und 
Spezifische Hashtags die von Partnern vorgegeben werden. 
Standard Seeding: ...

--- Retrieved doc 2 ---
Hashtags: Keine Hashtags 
Seeding: kein Seeding. ...

--- Retrieved doc 3 ---
Die meisten Jugendlichen/jungen Erwachsene sind auf Instagram aktiv. Das Posten von Events 
auf diesem Kanal ermöglicht der FWO ein gutes und interessantes Image für die FHNW in Olten 
zu erschaffen und somit zukünftige Studierende zu „überzeugen“ an der FH in Olten studieren zu ...



C:\Users\amuel\AppData\Local\Temp\ipykernel_32352\3188004710.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Hashtags für Instagram")


In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | FWO_PROMPT
#     | llm
#     | output_parser
# )

chain = (
{
    "context": retriever,
    "question": RunnablePassthrough(),
}
    | FWO_PROMPT
    | llm
    | StrOutputParser()
)


In [15]:
# from langchain_core.output_parsers import StrOutputParser

# USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

# result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
# print(result)

result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
print(result)

#result = chain.invoke(user_prompt)
#print(result)

**Kampagnenkonzept (Ziel – Kernbotschaft – Mehrwert)**  
- **Ziel:** Hohe Teilnahme‑ und Anmeldezahlen für die PubTour am 16. Oktober.  
- **Kernbotschaft:** Die PubTour verbindet Studierende, Fachpersonen und Praxispartner in lockerer Atmosphäre – ein Muss für Netzwerk und Spaß.  
- **Mehrwert:** Direkter Austausch, Einblicke in Unternehmen, kostenlose Fotoshoot‑Option und ein klarer Zeitplan, damit niemand etwas verpasst.

**Redaktionsplan (Phase – wiederkehrende Formate – Timing)**  
- **Phase:** „Reminder & Call‑to‑Action“ (letzte Woche vor dem Event).  
- **Formate:** Bild‑Post + Kurztext (Instagram, LinkedIn) – Text‑Nachricht + Bild (WhatsApp).  
- **Timing:** Veröffentlichung Dienstag, 08.10. um 10:00 Uhr; zweite Erinnerung Freitag, 12.10. um 16:00 Uhr.

---

## Instagram  

### Vorschlag 1 – Bild‑Post (Dienstag, 08.10., 10:00 Uhr)  
**Copy:**  
> 🎉 **PubTour 16. Oktober – Dein Netzwerk‑Boost!**  
> ⏰ Treffpunkt: 18:00 Uhr, Campus‑Café  
> 📍 Start: Hauptgebäude, Eingang A  
> 📸 

## UI

In [16]:
import gradio as gr

def answer(question: str) -> str:
    # Kein LangSmith, einfach direkt die Chain ausführen
    try:
        response = chain.invoke(question)
        return response
    except Exception as e:
        return f"⚠️ Fehler bei der Verarbeitung: {e}"

# --- Gradio UI ---
demo = gr.Interface(
    fn=answer,
    inputs=gr.Textbox(label="Question", placeholder="Type your question here..."),
    outputs=gr.Textbox(label="Answer", lines=10),
    title="FWO Chatbot",
    description="Write a social media post based on the context provided.",
)

# if __name__ == "__main__":
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>